In [29]:
import csv
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV

import matplotlib
from matplotlib import pyplot as plt


# from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
# from sklearn.linear_model import LogisticRegression,SGDClassifier
# from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from sklearn.metrics import accuracy_score


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
import pickle
data_features_spooky = open("pickled/data_features_spooky.pickle", "rb")
data= pickle.load(data_features_spooky)
data_features_spooky.close()


data.head()

,id,text,author,text_clean
0,id26305,"This process, however, afforded me no means of...",EAP,process howev afford mean ascertain dimens dun...
1,id17569,It never once occurred to me that the fumbling...,HPL,never onc occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand wa gold snuff box caper hill cut man...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,love spring look windsor terrac sixteen fertil...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,find noth els even gold superintend abandon at...


In [31]:
def cv(data):
    """
    Count vectorizing function.  Returns embedded vector and vectorizer.
    """
    count_vectorizer = CountVectorizer() # Bag-of-words vectorization
#     count_vectorizer = TfidfVectorizer() # Term Frequency-Inverse Document Frequency vectorization
    #count_vectorizer = CountVectorizer(ngram_range =(1, 2)) # n-gram vectorization  
    
    emb = count_vectorizer.fit_transform(data)
    features_names= count_vectorizer.get_feature_names()
    
    return emb, features_names

emb, dictionary = cv(data['text_clean'].tolist())

In [32]:
array = emb.todense()
emb2= pd.DataFrame(array)
emb2['output'] = data['author']
    

features = emb2.columns.tolist()
output = 'output'
# removing the output and the id from features
features.remove(output)

In [53]:
all_words = []

for w in data['text_clean'].tolist():
    all_words.append(w.lower())


In [ ]:
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

In [47]:
# word_features= dictionary

# def build_dictionary(emb):
#     features = {}
#     for w in word_features:
#         for i in emb:
#             features[w] = i
    
    
#     return features

# featuresets = [build_dictionary(j) for j in emb.todense()]
# # [(find_features(rev), category) for (rev, category) in documents]

In [48]:
# features={}
# for j in emb.todense():
#     for i in j:

#         for w in word_features:
#             features[w] = i

In [52]:
# testing_set= np.array(emb2[features])

In [49]:
# testing_set[features].reshape(1, -1)
# predicted = MNB_classifier.predict(testing_set[features].reshape(1, -1))

In [41]:
classifier_f = open("pickled/SentimentAnalysisTrainClassifierMine.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()


open_file = open("pickled/SentimentAnalysisTrainMNB_classifier.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("pickled/SentimentAnalysisTrainBernoulliNB_classifier.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


In [50]:
# predicted = MNB_classifier.predict(testing_set[features].reshape(1, -1))

In [42]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [43]:
voted_classifier = VoteClassifier(
                                  classifier,
#                                   LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier)


In [44]:
def sentiment(text):
    return voted_classifier(text)

In [45]:
voted_classifier(training_set[features].T[0])

TypeError: 'VoteClassifier' object is not callable

In [ ]:
voted_classifier.predict(training_set[features].T[0])

In [38]:
training_set[features].T[0]

0        0
1        0
2        0
3        0
4        0
        ..
14862    0
14863    0
14864    0
14865    0
14866    0
Name: 0, Length: 14867, dtype: int64